In [1]:
import joblib
from google.colab import files
import io
import pandas as pd
from io import BytesIO
import pickle
import requests
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate


In [30]:
# Importar el modelo generado
url = 'https://github.com/lsrinconc/tg-desercion/blob/1a49ab50452168c37ab21c7077ac52440d3c4fc9/modelo_knn.pkl?raw=true'
modelo_knn = joblib.load(BytesIO(requests.get(url).content))


In [34]:
#Subir el archivo csv con los estudiantes a predecir
uploaded = files.upload()
t=0
for fn in uploaded.keys():
  nombreArchivo=fn
  t=t+1

Saving predicciones.csv to predicciones (13).csv


In [35]:
if t != 1:
  print("ERROR: Se debe subir unicamente un archivo ")
elif not nombreArchivo.endswith('.csv'):
  print("ERROR: El archivo de estar en formato csv")
else:
  print('*** Inicio predicción ***')
  print('--- Formateando atributos')
  ds = pd.read_csv(io.BytesIO(uploaded[nombreArchivo]), sep=';')

  #Formateo de columnas
  #Formateo Edad_rango
  ds['Edad_rango'] = ds['edad_rango'].map( {'0 a 20': 16, 
                                          '21 a 25': 21,
                                          '26 a 30': 26, 
                                          '31 a 40': 31,
                                          'Mayor a 40': 40}).astype(int)
  # Formateo Genero 
  ds['Genero'] = ds['genero'].map( {'F': 0, 'M': 1}).astype(int)

  #Formateo Jornada
  ds['Jornada'] = ds['jornada'].map( {'DIURNA': 0, 'NOCTURNA': 1}).astype(int)

  #Formateo Estado civil
  ds['Estado_civil'] = ds['estado_civil'].map( {'CASADO': 0, 
                                                'DIVORCIADO': 1, 
                                                'SEPARADO': 2, 
                                                'SOLTERO': 3, 
                                                'UNION LIBRE': 4, 
                                                'OTRO':5}).astype(int)

  #Formateo ingresos familiares
  ds['Ingresos_familiares'] = ds['ingresos_familiares'].map( {'ENTRE 0-1 SALARIOS MINIMOS': 0, 
                                                              'ENTRE 1-2 SALARIOS MINIMOS': 1, 
                                                              'ENTRE 2-5 SALARIOS MINIMOS': 2, 
                                                              'MAYOR 5 SALARIOS  MINIMOS': 3}).astype(int)

  #Formateo calificaciones
  ds['C1_rango'] = ds['c1_rango'].map( {'ENTRE 0 Y 2.9': 0, 'ENTRE 3 Y 3.9': 3, 'ENTRE 4 Y 5': 5}).astype(int)
  ds['C2_rango'] = ds['c2_rango'].map( {'ENTRE 0 Y 2.9': 0, 'ENTRE 3 Y 3.9': 3, 'ENTRE 4 Y 5': 5}).astype(int)
  ds['P1_rango'] = ds['p1_rango'].map( {'ENTRE 0 Y 2.9': 0, 'ENTRE 3 Y 3.9': 3, 'ENTRE 4 Y 5': 5}).astype(int)
  ds['P2_rango'] = ds['p2_rango'].map( {'ENTRE 0 Y 2.9': 0, 'ENTRE 3 Y 3.9': 3, 'ENTRE 4 Y 5': 5}).astype(int)
  ds['Promedio_acumulado_rango'] = ds['promedio_acumulado_rango'].map( {'ENTRE 0 Y 2.9': 0, 'ENTRE 3 Y 3.9': 3, 'ENTRE 4 Y 5': 5}).astype(int)

  #Formateo promedio_asignaturas_sem_rango
  ds['Cantidad_asignaturas_sem_rango'] = ds['cantidad_asignaturas_sem_rango'].map( {'ENTRE 1 Y 4': 0, 'ENTRE 5 Y 8': 1, 'MAS DE 8': 2}).astype(int)


  # Array con los nombres de las columnas a eliminar
  columnas_eliminar = ['edad_rango','genero','jornada',
                  'estado_civil','ingresos_familiares',
                  'c1_rango','c2_rango','p1_rango','p2_rango',
                  'cantidad_asignaturas_sem_rango','promedio_acumulado_rango']
  # Se eliminan las columnas que sin formatear y las columnas que no son necesarias para el analisis y se guardan en una nueva variable
  ds_formateado = ds.drop(columnas_eliminar, axis = 1)
  print('--- Datos formateados correctamente')
  print('--- Realizando predicción\n')
  #Predicción utilizando el modelo con el algoritmo k vecinos más cercanos
  # Se obtiene la predicción de probabilidad

  prediccion_proba=modelo_knn.predict_proba(ds_formateado.values)
  print('Resuldatos de la predicción \n')
  print('Predicción de probabilidad - Matriz de probabilidad\n')
  print(prediccion_proba)
  # Se obtiene la predicción 
  prediccion=modelo_knn.predict(ds_formateado.values)
  print('Predicción\n')
  print(prediccion)
  print('\n')
  # Se reemplazan los valores de la predicción de numeros a cadena de texto
  prediccion= np.where(prediccion == 1, 'Desertor', 'No Desertor')
  # Se eliminan las probabilidades de la clase 0 
  prediccion_proba= np.delete(prediccion_proba,0, 1)
  # Se crea un rango de 1 a n para los estudiantes 
  rango = range(1,prediccion.size+1)
  #con base a la probabilidad de la clase 1 ( desertor ), se setea el nivel de riesgo
  conditions  = [ prediccion_proba[:,0] <.5, (prediccion_proba[:,0] >=.5) & (prediccion_proba[:,0] <.75), prediccion_proba[:,0] >=.75 ]
  choices     = [ "Bajo", 'Medio', 'Alto' ]
  nivel_riesgo= np.select(conditions, choices, default=np.nan)

  #Se crea la matriz con los datos de la predicción y nivel de riesgo
  tabla= np.array([rango,prediccion,nivel_riesgo],dtype=object)
  tabla = tabla.transpose()
  col_names = ["Estudiante","Clasificación", "Nivel riesgo deserción"]

  # Se dibuja la tabla con los resultado
  print('Tabulación resultados')
  print(tabulate(tabla, headers=col_names, tablefmt="fancy_grid"))
  print('\n*** Fin predicción ***')




*** Inicio predicción ***
--- Formateando atributos
--- Datos formateados correctamente
--- Realizando predicción

Resuldatos de la predicción 

Predicción de probabilidad - Matriz de probabilidad

[[0.8 0.2]
 [0.2 0.8]
 [0.3 0.7]]
Predicción

[0 1 1]


Tabulación resultados
╒══════════════╤═════════════════╤══════════════════════════╕
│   Estudiante │ Clasificación   │ Nivel riesgo deserción   │
╞══════════════╪═════════════════╪══════════════════════════╡
│            1 │ No Desertor     │ Bajo                     │
├──────────────┼─────────────────┼──────────────────────────┤
│            2 │ Desertor        │ Alto                     │
├──────────────┼─────────────────┼──────────────────────────┤
│            3 │ Desertor        │ Medio                    │
╘══════════════╧═════════════════╧══════════════════════════╛

*** Fin predicción ***
